In [ ]:
# TODO: The neural network
import tensorflow as tf
import matplotlib.pyplot as mb
tf.reset_default_graph()
from tensorflow.keras.datasets import mnist 

color_channels = 1
image_height = 28
image_width = 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)

class CNN:
    def __init__(self, image_height, image_width, channels, num_classes):
        self.input_= tf.placeholder(dtype = tf.float32, shape = [None, image_height, image_width, channels])      
        conv_layer_1 = tf.layers.conv2d(self.input_, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu)
        pooling_layer_1 = tf.layers.max_pooling2d(conv_layer_1, pool_size=[2, 2], strides=2)
        
        conv_layer_2 = tf.layers.conv2d(pooling_layer_1, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu)
        pooling_layer_2 = tf.layers.max_pooling2d(conv_layer_2, pool_size = [2, 2], strides = 2)
        
        conv_layer_3 = tf.layers.conv2d(pooling_layer_2, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu) 
        #pooling_layer_3 = tf.layers.max_pooling2d(conv_layer_3, pool_size = [2, 2], strides = 2)
        #conv_layer_4 = tf.layers.conv2d(conv_layer_3, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu) 
        #conv_layer_5 = tf.layers.conv2d(conv_layer_4, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu)
        #conv_layer_6 = tf.layers.conv2d(conv_layer_5, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu)
        #conv_layer_7 = tf.layers.conv2d(conv_layer_6, filters = 64, kernel_size = [2, 2], activation = tf.nn.relu)
        
        flattened_pooling = tf.layers.flatten(conv_layer_3)
        dense_layer = tf.layers.dense(flattened_pooling, 1024, activation = tf.nn.relu)
        dropout = tf.layers.dropout(dense_layer, rate = 0.4, training = True)
        
        
        outputs = tf.layers.dense(dropout, num_classes)
        self.choice = tf.argmax(outputs, axis=1)
        self.probability = tf.nn.softmax(outputs)
        self.labels = tf.placeholder(dtype = tf.float32, name = "labels")
        self.accuracy, self.accuracy_op = tf.metrics.accuracy(self.labels, self.choice)
        one_hot_labels = tf.one_hot(indices = tf.cast(self.labels, dtype = tf.int32), depth=num_classes)
        self.loss = tf.losses.softmax_cross_entropy(onehot_labels=one_hot_labels, logits = outputs)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0003)
        self.train_operation = optimizer.minimize(self.loss, global_step = tf.train.get_global_step())
        
        

In [ ]:
steps = 2000
#batch is the number of times the training example is used per step before updating
batch_size = 36
num_classes = 10
x_train = x_train.reshape(-1, image_height, image_width, 1)
x_test = x_test.reshape(-1, image_height, image_height, 1)
cnn = CNN(28, 28, 1, 10)
x_train = x_train.reshape(-1, image_height, image_width, 1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    step = 0
    count = 0
    while step < steps:        
        sess.run((cnn.train_operation, cnn.accuracy_op), feed_dict = {cnn.input_:x_train[step:step+batch_size], cnn.labels:y_train[step:step+batch_size]})
        step += batch_size
            
    nm = len(x_test)
    for i in range(0, nm):
        
        if i % 100 == 0:
            test_img = x_test[i]   
            test_img = test_img.reshape(28, 28)
            mb.imshow(test_img)
            mb.show()
            test_img = test_img.reshape(-1, 28, 28, 1)
            
        a = sess.run(cnn.choice, feed_dict = {cnn.input_:test_img})
        print(a)

        answer = y_test[i]
       
        #print(i)
        #a is the guess to the image
            
        if answer == a:
            count += 1
            
print("Number correct")
print(count)
print("Accuracy percentage")
print((count/nm)*100)